In [1]:
import pandas as pd
import numpy as np
import json

In [46]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import os

In [ ]:
next_month_perdiction_folder = '/home/amin/thesis/src/Step3_StockSelection/Final_result'

In [3]:
portfolio_path = '/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss'

In [4]:
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_LassoLogRegANN.json', 'r') as f :
    my_portfolio = json.load(f)

In [5]:
my_portfolio

{'2015-08-01': [['فباهنر.xlsx', -0.03986710963455149],
  ['خپارس.xlsx', -0.042105263157894736],
  ['ذوب.xlsx', -0.1],
  ['خزامیاد.xlsx', -0.1]],
 '2015-09-01': [['فولاژ.xlsx', -0.07103825136612021],
  ['شنفت.xlsx', -0.07651090342679127],
  ['شخارک.xlsx', 0.0308502024291498],
  ['شبندر.xlsx', -0.05167255915148219]],
 '2015-10-01': [['شراز.xlsx', -0.06570177109122072],
  ['فاسمین.xlsx', -0.0903866248693835],
  ['شیراز.xlsx', 0.04375569735642662],
  ['شاوان.xlsx', 0.08859743040685225]],
 '2015-11-01': [['فاسمین.xlsx', -0.045376220562894885],
  ['شاراک.xlsx', -0.045733518115224706],
  ['شبهرن.xlsx', -0.1],
  ['شپنا.xlsx', -0.1]],
 '2015-12-01': [['زاگرس.xlsx', 0.15230441518202945],
  ['شتران.xlsx', 0.35668003702560935],
  ['مارون.xlsx', 0.08277282918257287],
  ['فولاد.xlsx', 0.05390334572490706]],
 '2016-01-01': [['خزامیاد.xlsx', 0.1897463002114165],
  ['خراسان.xlsx', -0.044809044533296566],
  ['جم.xlsx', 0.02109053497942387],
  ['شراز.xlsx', 0.46503945678106073]],
 '2016-02-01': [['فخاس.x

In [6]:
daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'

In [28]:
next_month_perdiction_folder = '/home/amin/thesis/src/Step3_StockSelection/Final_result/LassoLogReg_ANN/LassoLogRegANN'

# LassoLogRegANN

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_LassoLogRegANN.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/LassoLogReg_ANN/LassoLogRegAnn_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_LassoLogRegAnn.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_LassoLogRegAnn.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
فباهنر.xlsx


خپارس.xlsx
ذوب.xlsx
خزامیاد.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.4000
Stock 3: 0.2000
Stock 4: 0.4000
Optimal Sharpe Ratio: 5.3631
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
فولاژ.xlsx
شنفت.xlsx
شخارک.xlsx
شبندر.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.4000
Stock 3: 0.2000
Stock 4: 0.0000
Optimal Sharpe Ratio: 5.8976
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
شراز.xlsx
فاسمین.xlsx
شیراز.xlsx
شاوان.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.0000
Stock 4: 0.4000
Optimal Sharpe Ratio: 4.8988
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
فاسمین.xlsx
شاراک.xlsx
شبهرن.xlsx
شپنا.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.0000
Stock 4: 0.4000
Optimal Sharpe Ratio: 4.8783
---------------------------------------------------------------------------

In [4]:
final_optimized_portfolio

{'2015-08-01': [[('فباهنر.xlsx', '0.000'),
   ('خپارس.xlsx', '0.400'),
   ('ذوب.xlsx', '0.200'),
   ('خزامیاد.xlsx', '0.400')]],
 '2015-09-01': [[('فولاژ.xlsx', '0.400'),
   ('شنفت.xlsx', '0.400'),
   ('شخارک.xlsx', '0.200'),
   ('شبندر.xlsx', '0.000')]],
 '2015-10-01': [[('شراز.xlsx', '0.400'),
   ('فاسمین.xlsx', '0.200'),
   ('شیراز.xlsx', '0.000'),
   ('شاوان.xlsx', '0.400')]],
 '2015-11-01': [[('فاسمین.xlsx', '0.400'),
   ('شاراک.xlsx', '0.200'),
   ('شبهرن.xlsx', '0.000'),
   ('شپنا.xlsx', '0.400')]],
 '2015-12-01': [[('زاگرس.xlsx', '0.400'),
   ('شتران.xlsx', '0.400'),
   ('مارون.xlsx', '0.200'),
   ('فولاد.xlsx', '0.000')]],
 '2016-01-01': [[('خزامیاد.xlsx', '0.179'),
   ('خراسان.xlsx', '0.249'),
   ('جم.xlsx', '0.287'),
   ('شراز.xlsx', '0.285')]],
 '2016-02-01': [[('فخاس.xlsx', '0.000'),
   ('شیراز.xlsx', '0.200'),
   ('ذوب.xlsx', '0.400'),
   ('فاسمین.xlsx', '0.400')]],
 '2016-03-01': [[('ارفع.xlsx', '0.092'),
   ('کرماشا.xlsx', '0.527'),
   ('شرانول.xlsx', '0.243'),
   ('شفن

In [5]:
best_sharpratio

{'2015-08-01': [('Optimal_Sharp_Ratio', '5.3631'), ('Risk_Free_Rate', 0.175)],
 '2015-09-01': [('Optimal_Sharp_Ratio', '5.8976'), ('Risk_Free_Rate', 0.175)],
 '2015-10-01': [('Optimal_Sharp_Ratio', '4.8988'), ('Risk_Free_Rate', 0.175)],
 '2015-11-01': [('Optimal_Sharp_Ratio', '4.8783'), ('Risk_Free_Rate', 0.175)],
 '2015-12-01': [('Optimal_Sharp_Ratio', '6.9808'), ('Risk_Free_Rate', 0.2452)],
 '2016-01-01': [('Optimal_Sharp_Ratio', 'nan'), ('Risk_Free_Rate', 0.2485)],
 '2016-02-01': [('Optimal_Sharp_Ratio', '13.2251'),
  ('Risk_Free_Rate', 0.2414)],
 '2016-03-01': [('Optimal_Sharp_Ratio', 'nan'), ('Risk_Free_Rate', 0.2965)],
 '2016-04-01': [('Optimal_Sharp_Ratio', '7.2698'), ('Risk_Free_Rate', 0.2195)],
 '2016-05-01': [('Optimal_Sharp_Ratio', 'nan'), ('Risk_Free_Rate', 0.2349)],
 '2016-06-01': [('Optimal_Sharp_Ratio', '9.1089'), ('Risk_Free_Rate', 0.24)],
 '2016-07-01': [('Optimal_Sharp_Ratio', '10.9491'),
  ('Risk_Free_Rate', 0.2354)],
 '2016-08-01': [('Optimal_Sharp_Ratio', 'nan'), (

# LassoLogRegRF

In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_LassoLogRegRF.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/LassoLogReg_RF/LassoLogRegRF_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_LassoLogRegRf.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_LassoLogRegRf.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شبریز.xlsx
خپارس.xlsx
شبندر.xlsx
Optimal Weights:
Stock 1: 0.2000
Stock 2: 0.4000
Stock 3: 0.4000
Optimal Sharpe Ratio: 6.0304
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
فاسمین.xlsx
شرانول.xlsx
ذوب.xlsx
شپدیس.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.0000
Optimal Sharpe Ratio: 5.8876
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
خپارس.xlsx
فخوز.xlsx
شپنا.xlsx
شیراز.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.0000
Optimal Sharpe Ratio: 4.7512
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
خراسان.xlsx
مارون.xlsx
شپدیس.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.4000
Stock 3: 0.2000
Optimal Sharpe Ratio: 10.1515
--------------------------------------------------------------------------------
Working on

# LassoLogRegSVM

In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_LassoLogRegSVM.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/LassoLogReg_SVM/LassoLogRegSvm_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_LassoLogRegSvm.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_LassoLogRegSvm.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شکبیر.xlsx
شبریز.xlsx
خودرو.xlsx
فایرا.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.0000
Stock 3: 0.2000
Stock 4: 0.4000
Optimal Sharpe Ratio: 6.1216
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
شپنا.xlsx
شکبیر.xlsx
خودرو.xlsx
شخارک.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.4000
Optimal Sharpe Ratio: 8.0224
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
ارفع.xlsx
شکبیر.xlsx
خساپا.xlsx
شپنا.xlsx
Optimal Weights:
Stock 1: 0.2102
Stock 2: 0.2956
Stock 3: 0.1749
Stock 4: 0.3194
Optimal Sharpe Ratio: nan
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
شخارک.xlsx
خودرو.xlsx
شکبیر.xlsx
ارفع.xlsx
Optimal Weights:
Stock 1: 0.0668
Stock 2: 0.4070
Stock 3: 0.2643
Stock 4: 0.2619
Optimal Sharpe Ratio: nan
-----------------------------------------------

# LassoLogRegXGB

In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_LassoLogRegXGB.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/LassoLogReg_XGB/LassoLogRegXGB_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_LassoLogRegXgb.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_LassoLogRegXgb.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شبریز.xlsx
کچاد.xlsx
شاوان.xlsx
خراسان.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.0000
Stock 3: 0.4000
Stock 4: 0.2000
Optimal Sharpe Ratio: 6.2058
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
فباهنر.xlsx
زاگرس.xlsx
شیران.xlsx
شبهرن.xlsx
Optimal Weights:
Stock 1: 0.2000
Stock 2: 0.4000
Stock 3: 0.0000
Stock 4: 0.4000
Optimal Sharpe Ratio: 9.0587
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
کگل.xlsx
شیراز.xlsx
خراسان.xlsx
کچاد.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.0000
Optimal Sharpe Ratio: 10.9055
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
شفن.xlsx
مارون.xlsx
شپدیس.xlsx
شیراز.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.4000
Optimal Sharpe Ratio: 11.4292
--------------------------------------

# RF-ANN

In [9]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_RFANN.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/RF-ANN/RfAnn_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_RfAnn.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_RfAnn.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
فباهنر.xlsx
شکبیر.xlsx
شبهرن.xlsx
خساپا.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.4000
Optimal Sharpe Ratio: 6.9090
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
فولاژ.xlsx
خپارس.xlsx
فولاد.xlsx
شرانول.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.4000
Stock 3: 0.4000
Stock 4: 0.2000
Optimal Sharpe Ratio: 5.6649
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
شبهرن.xlsx
فباهنر.xlsx
شخارک.xlsx
خساپا.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.0000
Stock 4: 0.4000
Optimal Sharpe Ratio: 7.5098
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
خساپا.xlsx
شفن.xlsx
شاوان.xlsx
کرماشا.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.0000
Stock 3: 0.4000
Stock 4: 0.2000
Optimal Sharpe Ratio: 5.3724
-----------------------------------

# RF - RF

In [10]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_RFRF.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/RF-RF/RfRf_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_RfRf.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_RfRf.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شراز.xlsx
خودرو.xlsx
شبندر.xlsx
شپدیس.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.0000
Optimal Sharpe Ratio: 6.6466
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
شپدیس.xlsx
خراسان.xlsx
شپنا.xlsx
شرانول.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.4000
Optimal Sharpe Ratio: 5.4599
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
مارون.xlsx
ذوب.xlsx
فایرا.xlsx
Optimal Weights:
Stock 1: 0.2000
Stock 2: 0.4000
Stock 3: 0.4000
Optimal Sharpe Ratio: 7.9664
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
فایرا.xlsx
کرماشا.xlsx
مارون.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.4000
Stock 3: 0.2000
Optimal Sharpe Ratio: 10.3386
--------------------------------------------------------------------------------
Working on

# RF - SVM

In [11]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_RFSVM.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/RF-SVM/RfSvm_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_RfSvm.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_RfSvm.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شپدیس.xlsx
فاسمین.xlsx
شنفت.xlsx
شکبیر.xlsx
Optimal Weights:
Stock 1: 0.0000
Stock 2: 0.4000
Stock 3: 0.4000
Stock 4: 0.2000
Optimal Sharpe Ratio: 6.7974
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
شپدیس.xlsx
مارون.xlsx
جم.xlsx
خراسان.xlsx
Optimal Weights:
Stock 1: 0.3485
Stock 2: 0.0573
Stock 3: 0.3548
Stock 4: 0.2395
Optimal Sharpe Ratio: nan
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
شبریز.xlsx
مارون.xlsx
جم.xlsx
شنفت.xlsx
Optimal Weights:
Stock 1: 0.1767
Stock 2: 0.0666
Stock 3: 0.2040
Stock 4: 0.5527
Optimal Sharpe Ratio: nan
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
شپنا.xlsx
شراز.xlsx
جم.xlsx
Optimal Weights:
Stock 1: 0.2479
Stock 2: 0.4760
Stock 3: 0.2761
Optimal Sharpe Ratio: nan
--------------------------------------------------------------------------------
Wor

# RF - XGB

In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import json
import os
import warnings

warnings.filterwarnings('ignore')





daily_data_folder = '/home/amin/thesis/src/data/MAIN-FILES/FINNAL_DATA2'
with open ('/home/amin/thesis/src/Step4_Portfolio_Optimization/Set_StopLoss/my__FINAL_portfolio_return_RFXGB.json', 'r') as f : #1
    my_portfolio = json.load(f)


final_optimized_portfolio = {}
best_sharpratio = {}

for date, value in my_portfolio.items() :

    print(f'Working on {date} ...')


    date_range = pd.date_range(start='2013-09-01', end=date)
    df = pd.DataFrame(date_range, columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])

    predicted_return_df = pd.DataFrame()



    for stock in value :
        stock_name = stock[0]
        stock_return = stock[1]
        print(stock_name)
        stock_daily_path = os.path.join(daily_data_folder, stock_name)
        stock_df = pd.read_excel(stock_daily_path)
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        stock_df = stock_df[(stock_df['Date'] >= '2013-09-01') & (stock_df['Date'] <= date)]
        stock_df['Daily_Return'] = stock_df['پایانی*سهم*'].pct_change()

        #RISK FREE
        rf_df = stock_df[['Date', 'نرخ بدون ریسک*کلان*']]
        rf_df.rename(columns={'نرخ بدون ریسک*کلان*' : 'RiskFree'} , inplace=True)
        rf_df['RiskFree'] = rf_df['RiskFree'].replace(0, 0.175)
        rf_df2 =rf_df.copy()
        rf_df2.set_index('Date', inplace=True)
        rf_last_month = rf_df2.resample('M').mean().iloc[-1]['RiskFree']


        #Daily RETURN
        return_df = stock_df[['Date','Daily_Return']]



        # Step1 - Creating The Daily Return Data Frame
        df = pd.merge(df, return_df, on='Date', how='outer')
        df.fillna(method='bfill', inplace=True)
        df_col = df.columns
        df.rename(columns={df_col[-1] : f'{stock_name}'} , inplace=True)



        # Next Month Prediction :
        p = pd.read_excel(f'/home/amin/thesis/src/Step3_StockSelection/Final_result/RF-XGB/RfXgb_{stock_name}')  #2
        p.drop('Unnamed: 0', axis=1, inplace=True)
        p.columns = pd.to_datetime(p.columns, format='%Y-%m')
        p.columns = p.columns.strftime('%Y-%m-%d')
        Next_Month_Prediction = int(p[date].iloc[-1].replace('[', '').replace(']', '').replace(".", ''))
        if Next_Month_Prediction == 1 :
            Next_Month_Prediction = 0.025
        elif Next_Month_Prediction == 2 :
            Next_Month_Prediction = 0.050
        elif Next_Month_Prediction == 3 :
            Next_Month_Prediction = 0.075
        predicted_return_df[stock_name] = [Next_Month_Prediction]


    import pandas as pd
    import numpy as np
    from scipy.optimize import minimize

    # Calculate the covariance matrix
    returns = df.drop('Date', axis=1)
    cov_matrix = returns.cov()
    predicted_returns = predicted_return_df.values

    # Number of stocks
    num_stocks = len(returns.columns)

    # Objective function to maximize Sharpe ratio using predicted returns
    def objective(weights, cov_matrix, predicted_returns, risk_free_rate):
        # Expected portfolio return using predicted returns
        portfolio_return = np.dot(predicted_returns, weights)

        # Portfolio standard deviation
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_stddev

        return -sharpe_ratio  # We use negative Sharpe ratio because scipy's minimize function minimizes the objective function



    max_weight_constraint = 0.40
    # Constraints
    constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'ineq', 'fun': lambda weights: max_weight_constraint - weights},  # Maximum weight constraint
]

    # Bounds for each weight (between 0 and 1)
    bounds = tuple((0, 1) for _ in range(num_stocks))

    # Initial equal weights
    #initial_weights = np.full(num_stocks, 1.0 / num_stocks)

    initial_weights = np.random.rand(num_stocks)
    initial_weights /= initial_weights.sum()




    last_day_risk_free_rate = rf_last_month



    # Run optimization to maximize Sharpe ratio
    result = minimize(objective, initial_weights, args=(cov_matrix, predicted_returns, last_day_risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

    # Optimal weights
    optimal_weights = result.x

    # Display the optimal weights
    print("Optimal Weights:")
    for i, weight in enumerate(optimal_weights):
        print(f"Stock {i+1}: {weight:.4f}")

    # Display the optimal Sharpe ratio
    optimal_portfolio_return = np.dot(predicted_returns, optimal_weights)
    optimal_portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
    optimal_sharpe_ratio = (optimal_portfolio_return - last_day_risk_free_rate) / optimal_portfolio_stddev

    optimal_sharpe_ratio = f"{abs(optimal_sharpe_ratio[0]):.4f}"


    print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio}")
    print('----------------' * 5)
    best_sharpratio[date] = [('Optimal_Sharp_Ratio',optimal_sharpe_ratio),('Risk_Free_Rate', last_day_risk_free_rate)]
    list2 = []
    for i in range(len(optimal_weights)) :
        list2.append((returns.columns[i],f'{optimal_weights[i]:.3f}'))
    final_optimized_portfolio[date] = [list2]


with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/OptimizedPortfolio_weight_RfXgb.json', 'w') as f : #3
    json.dump(final_optimized_portfolio, f)

with open('/home/amin/thesis/src/Step4_Portfolio_Optimization/StockWeight_SharpRatio/Sharpe_Ratio_Portfolio_RfXgb.json', 'w')  as k :      #4
    json.dump(best_sharpratio, k)












Working on 2015-08-01 ...
شراز.xlsx
شبهرن.xlsx
شبریز.xlsx
شفن.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.4000
Stock 4: 0.0000
Optimal Sharpe Ratio: 8.1008
--------------------------------------------------------------------------------
Working on 2015-09-01 ...
شراز.xlsx
شبهرن.xlsx
شپدیس.xlsx
خساپا.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.2000
Stock 3: 0.0000
Stock 4: 0.4000
Optimal Sharpe Ratio: 8.7331
--------------------------------------------------------------------------------
Working on 2015-10-01 ...
شیراز.xlsx
شپدیس.xlsx
خراسان.xlsx
شبهرن.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.0000
Stock 3: 0.2000
Stock 4: 0.4000
Optimal Sharpe Ratio: 9.9310
--------------------------------------------------------------------------------
Working on 2015-11-01 ...
خودرو.xlsx
شپدیس.xlsx
ذوب.xlsx
شفن.xlsx
Optimal Weights:
Stock 1: 0.4000
Stock 2: 0.0000
Stock 3: 0.4000
Stock 4: 0.2000
Optimal Sharpe Ratio: 10.5670
-------------------------------------------